## Computing CLEVER metric with CNN model from Keras trained with CIFAR10 dataset

In [100]:
# import neccessary files
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import tensorflow as tf

# art library
from art import metrics
from art.attacks import DeepFool
from art.classifiers import KerasClassifier
from art.utils import load_dataset


## Preparing dataset and environment
In this file, we will use the CIFAR10 dataset.
We will disable tensorflow 2.0's eager evaluation as ART has not been updated to support eager evaluation.

# disable tensorflow 2.0 eager evaluation as it is not yet supported
tf.compat.v1.disable_eager_execution()
# Read CIFAR10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str('cifar10'))
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]
im_shape = x_train[0].shape

## Creating and preparing the model
For this example, we will use Keras Convolutional Neural Network model (CNN) without any adversarial defense training.

In [101]:
# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Wrapping a whitebox classifier
This creates a classifier wrapper for training of model

In [30]:
# Create classifier wrapper
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)

Epoch 1/10
40/39 [==============================] - 3s 75ms/step - loss: 2.1691 - accuracy: 0.1854
Epoch 2/10
40/39 [==============================] - 3s 72ms/step - loss: 1.8881 - accuracy: 0.3100
Epoch 3/10
40/39 [==============================] - 3s 72ms/step - loss: 1.7466 - accuracy: 0.3678
Epoch 4/10
40/39 [==============================] - 3s 72ms/step - loss: 1.6204 - accuracy: 0.4041
Epoch 5/10
40/39 [==============================] - 3s 71ms/step - loss: 1.5235 - accuracy: 0.4473
Epoch 6/10
40/39 [==============================] - 3s 71ms/step - loss: 1.4455 - accuracy: 0.4787
Epoch 7/10
40/39 [==============================] - 3s 71ms/step - loss: 1.3619 - accuracy: 0.5102
Epoch 8/10
40/39 [==============================] - 3s 72ms/step - loss: 1.2992 - accuracy: 0.5316
Epoch 9/10
40/39 [==============================] - 3s 72ms/step - loss: 1.2059 - accuracy: 0.5646
Epoch 10/10
40/39 [==============================] - 3s 71ms/step - loss: 1.1821 - accuracy: 0.5777


## Computing of CLEVER score
```metrics.clever_u``` is used to evaluate CLEVER score for untargetted attacks.
```metrics.clever_t``` is used to evaluate CLEVER score for targetted attacks.
#### Usage
##### official documentation: <a href="https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/metrics.html">click here</a> or <a href="https://arxiv.org/pdf/1807.01069.pdf"> click here</a>

```metrics.clever_u(classifier, x, nb_batches, batch_size, radius, norm, c_init=1, pool_factor=10)```

<ul>
    <li><b>classifier</b> (classifier) - classifier object we wrapped above</li>
    <li><b>x</b> (np.ndarray) - input sample (typically use x_test)</li>
    <li><b>nb_batches</b> (int) - Number of repetitions to estimate CLEVER</li>
    <li><b>batch_size</b> (int) - Number of random examples to sample per batch</li>
    <li><b>radius</b> (float) - ball of radius of the maximum perturbation</li>
    <li><b>norm</b> (int) - norm of gradient x (current support by ART: 1,2,np.inf</li>
    <li><b>c_init</b> (float) – initialization of Weibull distribution (default=1)</li>
    <li><b>pool_factor</b> (int) – The factor to create a pool of random samples with size pool_factor x n_s (default=10)</li>
</ul>


In [31]:
#using CLEVER score with first 9 test samples
scores = []
for i in range(9):
    scores.append(metrics.clever_u(classifier,x_test[i],10,10,50,1))

avg_score = sum(scores)/len(scores)
print(avg_score)

2.3125193516440703


## BlackBox testing
We can only access 2 functions in the blackbox model:
 - `classifierBlack.predict(x_test)` returns predicted output from x_test
 - `classifierBlack.nb_classes()` returns number of target classes 


### Training the created CNN model above and preparing classifier

In [171]:
#training the CNN model created above and defining the predict function
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

#train the model above
model.fit(x_train, y_train, batch_size = 128, epochs = 10)

#create a function to predict one hot encoded inputs using trained model
def classify_one_hot(input):
    data = model.predict_classes(input)
    data = array(data)
    # one hot encode
    encoded = to_categorical(data, num_classes=10)
    return encoded

Epoch 1/10
5000/5000 [==============================] - 5s 904us/step - loss: 0.4450 - accuracy: 0.8488
Epoch 2/10
5000/5000 [==============================] - 5s 927us/step - loss: 0.3509 - accuracy: 0.8776
Epoch 3/10
5000/5000 [==============================] - 4s 880us/step - loss: 0.3009 - accuracy: 0.8974
Epoch 4/10
5000/5000 [==============================] - 4s 878us/step - loss: 0.3115 - accuracy: 0.8894
Epoch 5/10
5000/5000 [==============================] - 4s 881us/step - loss: 0.2716 - accuracy: 0.9048
Epoch 6/10
5000/5000 [==============================] - 4s 887us/step - loss: 0.2492 - accuracy: 0.9116
Epoch 7/10
5000/5000 [==============================] - 4s 888us/step - loss: 0.2545 - accuracy: 0.9144
Epoch 8/10
5000/5000 [==============================] - 4s 894us/step - loss: 0.3034 - accuracy: 0.8956
Epoch 9/10
5000/5000 [==============================] - 4s 880us/step - loss: 0.2746 - accuracy: 0.9006
Epoch 10/10
5000/5000 [==============================] - 5s 901u

### Wrapping a Blackbox classifier

In [172]:
#create a blackbox wrapper
from art.classifiers import BlackBoxClassifier
classifierBlack = BlackBoxClassifier(classify_one_hot,x_train.shape[1:],10)

In [173]:
classifierBlack.predict(x_test[:3])

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [176]:
from art.attacks import FastGradientMethod
FGMAttack = FastGradientMethod(classifierBlack,1)

TypeError: For `FastGradientMethod` classifier must be an instance of `art.classifiers.classifier.ClassifierGradients`, the provided classifier is instance of (<class 'art.classifiers.classifier.Classifier'>,).  The classifier needs to provide gradients.